In [6]:
import pandas as pd
import numpy as np

In [1]:
import torch
from torch import nn

class DecompositionLayer(nn.Module):
    """
    Returns the trend and the seasonal parts of the time series.
    """

    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=1, padding=0) # moving average

    def forward(self, x):
        """Input shape: Batch x Time x EMBED_DIM"""
        # padding on the both ends of time series
        num_of_pads = (self.kernel_size - 1) // 2
        front = x[:, 0:1, :].repeat(1, num_of_pads, 1)
        end = x[:, -1:, :].repeat(1, num_of_pads, 1)
        x_padded = torch.cat([front, x, end], dim=1)

        # calculate the trend and seasonal part of the series
        x_trend = self.avg(x_padded.permute(0, 2, 1)).permute(0, 2, 1)
        x_seasonal = x - x_trend
        return x_seasonal, x_trend

c:\Users\atuli\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [2]:
import torch

def autocorrelation(query_states, key_states):
    """
    Computes autocorrelation(Q,K) using `torch.fft`.
    Think about it as a replacement for the QK^T in the self-attention.

    Assumption: states are resized to same shape of [batch_size, time_length, embedding_dim].
    """
    query_states_fft = torch.fft.rfft(query_states, dim=1)
    key_states_fft = torch.fft.rfft(key_states, dim=1)
    attn_weights = query_states_fft * torch.conj(key_states_fft)
    attn_weights = torch.fft.irfft(attn_weights, dim=1)

    return attn_weights

In [3]:
import torch
import math

def time_delay_aggregation(attn_weights, value_states, autocorrelation_factor=2):
    """
    Computes aggregation as value_states.roll(delay) * top_k_autocorrelations(delay).
    The final result is the autocorrelation-attention output.
    Think about it as a replacement of the dot-product between attn_weights and value states.

    The autocorrelation_factor is used to find top k autocorrelations delays.
    Assumption: value_states and attn_weights shape: [batch_size, time_length, embedding_dim]
    """
    bsz, num_heads, tgt_len, channel = ...
    time_length = value_states.size(1)
    autocorrelations = attn_weights.view(bsz, num_heads, tgt_len, channel)

    top_k = int(autocorrelation_factor * math.log(time_length))
    autocorrelations_mean = torch.mean(autocorrelations, dim=(1, -1))  # bsz x tgt_len
    top_k_autocorrelations, top_k_delays = torch.topk(autocorrelations_mean, top_k, dim=1)

    top_k_autocorrelations = torch.softmax(top_k_autocorrelations, dim=-1)  

    delays_agg = torch.zeros_like(value_states).float()  
    for i in range(top_k):
        value_states_roll_delay = value_states.roll(shifts=-int(top_k_delays[i]), dims=1)
        top_k_at_delay = top_k_autocorrelations[:, i]
        top_k_resized = top_k_at_delay.view(-1, 1, 1).repeat(num_heads, tgt_len, channel)
        delays_agg += value_states_roll_delay * top_k_resized

    attn_output = delays_agg.contiguous()
    return attn_output

In [8]:
dailyEnergy = pd.read_excel('dailyEnergyWithFeatures.xlsx')
dailyEnergy['weekday'] = dailyEnergy['TimeReviewed'].dt.dayofweek
dailyEnergy['day_type'] = np.zeros(len(dailyEnergy))

dailyEnergy['day_type'][(dailyEnergy['weekday']==5)|(dailyEnergy['weekday']==6)|(dailyEnergy['TimeReviewed']=='2017-12-26')|(dailyEnergy['TimeReviewed']=='2018-1-1')|(dailyEnergy['TimeReviewed']=='2018-1-14')|(dailyEnergy['TimeReviewed']=='2018-1-26')|(dailyEnergy['TimeReviewed']=='2018-5-1')|(dailyEnergy['TimeReviewed']=='2018-8-15')|(dailyEnergy['TimeReviewed']=='2018-10-2')|(dailyEnergy['TimeReviewed']=='2018-12-25')|(dailyEnergy['TimeReviewed']=='2019-1-1')|(dailyEnergy['TimeReviewed']=='2019-1-14')|(dailyEnergy['TimeReviewed']=='2019-1-26')|(dailyEnergy['TimeReviewed']=='2019-5-1')]=1
dailyEnergy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   TimeReviewed                 505 non-null    datetime64[ns]
 1   Solar_Power_Consumption(Kw)  505 non-null    float64       
 2   Temp( C)                     505 non-null    float64       
 3   EmployeeCount                505 non-null    int64         
 4   weekday                      505 non-null    int32         
 5   day_type                     505 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(1)
memory usage: 21.8 KB


C:\Users\atuli\AppData\Local\Temp\ipykernel_13104\1367160064.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dailyEnergy['day_type'][(dailyEnergy['weekday']==5)|(dailyEnergy['weekday']==6)|(dailyEnergy['TimeReviewed']=='2017-12-26')|(dai

In [9]:
dailyEnergy

,TimeReviewed,Solar_Power_Consumption(Kw),Temp( C),EmployeeCount,weekday,day_type
0,2017-12-26,12782.411988,25.000000,121,1,1.0
1,2017-12-27,13883.010020,25.375000,0,2,0.0
2,2017-12-28,13766.083029,24.833333,0,3,0.0
3,2017-12-29,12904.092990,24.833333,0,4,0.0
4,2017-12-30,7985.972000,25.906250,0,5,1.0
...,...,...,...,...,...,...
500,2019-05-10,15619.215017,27.041667,184,4,0.0
501,2019-05-11,9179.519970,26.662921,166,5,1.0
502,2019-05-12,0.000000,29.000000,11,6,1.0
503,2019-05-13,12396.665989,27.169492,186,0,0.0


In [63]:
df_elect = dailyEnergy[["Temp( C)", "EmployeeCount", "weekday","day_type", 'Solar_Power_Consumption(Kw)']]

elect_train = pd.DataFrame(data=df_elect.head(350))
elect_test = pd.DataFrame(data=df_elect.head(155))

XX_elect_train = elect_train.drop('Solar_Power_Consumption(Kw)', axis = 1).reset_index().drop('index', axis = 1)
XX_elect_test = elect_test.drop('Solar_Power_Consumption(Kw)', axis = 1).reset_index().drop('index', axis = 1)

YY_elect_train = elect_train[['Solar_Power_Consumption(Kw)']]
YY_elect_test = elect_test[['Solar_Power_Consumption(Kw)']]

In [34]:
len(YY_elect_train.values)

350

In [12]:
YY_elect_train

,Solar_Power_Consumption(Kw)
0,12782.411988
1,13883.010020
2,13766.083029
3,12904.092990
4,7985.972000
...,...
345,9587.865992
346,9994.493010
347,8061.357007
348,997.568998


In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class SolarPowerDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        x = torch.tensor(sample.drop('Solar_Power_Consumption(Kw)').values, dtype=torch.float32)
        y = torch.tensor(sample['Solar_Power_Consumption(Kw)'], dtype=torch.float32)
        return x, y

dailyEnergy = pd.read_excel('dailyEnergyWithFeatures.xlsx')

dailyEnergy['weekday'] = dailyEnergy['TimeReviewed'].dt.dayofweek
dailyEnergy['day_type'] = np.zeros(len(dailyEnergy))
holidays = ['2017-12-26', '2018-1-1', '2018-1-14', '2018-1-26', '2018-5-1', '2018-8-15', '2018-10-2', '2018-12-25', '2019-1-1', '2019-1-14', '2019-1-26', '2019-5-1']
dailyEnergy.loc[dailyEnergy['TimeReviewed'].isin(holidays), 'day_type'] = 1

df_elect = dailyEnergy[["Temp( C)", "EmployeeCount", "weekday", "day_type", 'Solar_Power_Consumption(Kw)']]

train_size = 350
test_size = 155
elect_train = df_elect.head(train_size)
elect_train
elect_test = df_elect.tail(test_size)

train_dataset = SolarPowerDataset(elect_train)
test_dataset = SolarPowerDataset(elect_test)

class ForeCaster(nn.Module):
    def __init__(self):
        super().__init__()
        self.decomposition = DecompositionLayer(kernel_size=3)
        self.fc = nn.Linear(4, 1)  

    def forward(self, x):
        seasonal, trend = self.decomposition(x)
        print("Seasonal shape:", seasonal.shape)
        print("Trend shape:", trend.shape)        
        seasonal_trend = torch.cat((seasonal, trend), dim=1)
        print("Seasonal and trend concatenated shape:", seasonal_trend.shape)  
        output = self.fc(seasonal_trend)
        return output

model = ForeCaster()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for x_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        print("Input shape:", x_batch.shape)
        output = model(x_batch)
#         loss = loss_fn(output.squeeze(), y_batch)
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
    
#     print(f"Epoch {epoch+1}, Train Loss: {total_loss / len(train_dataloader)}")

# model.eval()
# test_loss = 0
# with torch.no_grad():
#     for x_batch, y_batch in test_dataloader:
#         print("Input shape:", x_batch.shape)
#         output = model(x_batch)
#         loss = loss_fn(output.squeeze(), y_batch)
#         test_loss += loss.item()

# print(f"Test Loss: {test_loss / len(test_dataloader)}")


Input shape: torch.Size([32, 4])


C:\Users\atuli\AppData\Local\Temp\ipykernel_13104\3633244058.py:24: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  dailyEnergy.loc[dailyEnergy['TimeReviewed'].isin(holidays), 'day_type'] = 1


IndexError: too many indices for tensor of dimension 2

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler


In [68]:
X_train = XX_elect_train
y_train = YY_elect_train
X_test = XX_elect_test
y_test = YY_elect_test

In [48]:
from tensorflow import keras
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.6):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)

        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout2 = layers.Dropout(rate)

        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout3 = layers.Dropout(rate)

        self.layernorm4 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout4 = layers.Dropout(rate)
        
def call(self, inputs, training):
    attn_output = self.att(inputs, inputs)
    attn_output = self.dropout1(attn_output, training=training)

    out1 = self.layernorm1(inputs + attn_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)

    out2 = self.layernorm2(out1 + ffn_output)

    ffn_output2 = self.ffn(out2)
    ffn_output2 = self.dropout3(ffn_output2, training=training)

    out3 = self.layernorm3(out2 + ffn_output2)

    ffn_output3 = self.ffn(out3)
    ffn_output3 = self.dropout4(ffn_output3, training=training)  

    return self.layernorm4(out3 + ffn_output3)

In [73]:
embed_dim = 1
num_heads = 6
ff_dim = 64 # related to feed forward network
num_blocks = 10

In [70]:
X_train.shape

(350, 4)

In [79]:
def transformer_model():
    inputs = layers.Input(shape=(X_train.shape[1],))
    x = layers.Reshape(target_shape=(4, 1))(inputs)
    transformer_blocks = [TransformerBlock(embed_dim, num_heads, ff_dim) for _ in range(num_blocks)]
    
    for transformer_block in transformer_blocks:
        x = transformer_block(x, training=True)

    x = layers.Flatten()(x)

    x = layers.Dense(90, activation="relu")(x)
    x = layers.Dropout(0.6)(x)

    x = layers.Dense(60, activation="relu")(x)
    x = layers.Dropout(0.6)(x)

    x = layers.Dense(30, activation="relu")(x)
    x = layers.Dropout(0.6)(x)
    
    outputs = layers.Dense(1)(x)

    model = keras.Model(inputs, outputs)
    return model

model = transformer_model()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [80]:
history = model.fit(X_train, y_train, epochs=25, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 90s 189ms/step - accuracy: 0.0151 - loss: 109857136.0000 - val_accuracy: 0.0129 - val_loss: 122957824.0000
Epoch 2/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.0176 - loss: 111502392.0000 - val_accuracy: 0.0129 - val_loss: 122957400.0000
Epoch 3/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.0114 - loss: 114231000.0000 - val_accuracy: 0.0129 - val_loss: 122956976.0000
Epoch 4/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.0242 - loss: 116439792.0000 - val_accuracy: 0.0129 - val_loss: 122956512.0000
Epoch 5/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.0114 - loss: 111284416.0000 - val_accuracy: 0.0129 - val_loss: 122956096.0000
Epoch 6/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.0227 - loss: 111289232.0000 - val_accuracy: 0.0129 - val_loss: 122955656.0000
Epoch 7/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.0136 - loss: 111603280.0000 - val_accuracy: 0.0129 - val_loss: 122955232.00